In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import permutations
import json, os
from collections import OrderedDict
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb

from matplotlib import pyplot

In [2]:
pd.set_option('display.max_columns', None)

df = pd.read_csv('../../BPIC_Classification/processed_dataset/P_index_excat_12.csv')
df.head()

,Variant_0,(case) AMOUNT_REQ_0,concept:name_0,lifecycle:transition_0,Variant_1,(case) AMOUNT_REQ_1,concept:name_1,lifecycle:transition_1,Variant_2,(case) AMOUNT_REQ_2,concept:name_2,lifecycle:transition_2,Variant_3,(case) AMOUNT_REQ_3,concept:name_3,lifecycle:transition_3,Variant_4,(case) AMOUNT_REQ_4,concept:name_4,lifecycle:transition_4,Variant_5,(case) AMOUNT_REQ_5,concept:name_5,lifecycle:transition_5,Variant_6,(case) AMOUNT_REQ_6,concept:name_6,lifecycle:transition_6,Variant_7,(case) AMOUNT_REQ_7,concept:name_7,lifecycle:transition_7,Variant_8,(case) AMOUNT_REQ_8,concept:name_8,lifecycle:transition_8,Variant_9,(case) AMOUNT_REQ_9,concept:name_9,lifecycle:transition_9,Activity_0_A_SUBMITTED-COMPLETE,Activity_1_A_PARTLYSUBMITTED-COMPLETE,Activity_2_A_DECLINED-COMPLETE,Activity_2_A_PREACCEPTED-COMPLETE,Activity_2_W_Afhandelen leads-SCHEDULE,Activity_2_W_Beoordelen fraude-SCHEDULE,Activity_3_W_Afhandelen leads-START,Activity_3_W_Beoordelen fraude-START,Activity_3_W_Completeren aanvraag-SCHEDULE,Activity_4_A_DECLINED-COMPLETE,Activity_4_A_PREACCEPTED-COMPLETE,Activity_4_W_Afhandelen leads-COMPLETE,Activity_4_W_Afhandelen leads-SCHEDULE,Activity_4_W_Beoordelen fraude-COMPLETE,Activity_4_W_Beoordelen fraude-SCHEDULE,Activity_4_W_Completeren aanvraag-COMPLETE,Activity_4_W_Completeren aanvraag-START,Activity_5_A_ACCEPTED-COMPLETE,Activity_5_A_CANCELLED-COMPLETE,Activity_5_A_DECLINED-COMPLETE,Activity_5_W_Afhandelen leads-COMPLETE,Activity_5_W_Afhandelen leads-START,Activity_5_W_Beoordelen fraude-COMPLETE,Activity_5_W_Beoordelen fraude-START,Activity_5_W_Completeren aanvraag-COMPLETE,Activity_5_W_Completeren aanvraag-SCHEDULE,Activity_6_A_CANCELLED-COMPLETE,Activity_6_A_DECLINED-COMPLETE,Activity_6_A_FINALIZED-COMPLETE,Activity_6_A_PREACCEPTED-COMPLETE,Activity_6_O_SELECTED-COMPLETE,Activity_6_W_Afhandelen leads-COMPLETE,Activity_6_W_Afhandelen leads-SCHEDULE,Activity_6_W_Afhandelen leads-START,Activity_6_W_Beoordelen fraude-COMPLETE,Activity_6_W_Beoordelen fraude-SCHEDULE,Activity_6_W_Beoordelen fraude-START,Activity_6_W_Completeren aanvraag-COMPLETE,Activity_6_W_Completeren aanvraag-START,Activity_7_A_ACCEPTED-COMPLETE,Activity_7_A_CANCELLED-COMPLETE,Activity_7_A_DECLINED-COMPLETE,Activity_7_A_FINALIZED-COMPLETE,Activity_7_O_SELECTED-COMPLETE,Activity_7_W_Afhandelen leads-COMPLETE,Activity_7_W_Afhandelen leads-SCHEDULE,Activity_7_W_Afhandelen leads-START,Activity_7_W_Beoordelen fraude-COMPLETE,Activity_7_W_Beoordelen fraude-SCHEDULE,Activity_7_W_Beoordelen fraude-START,Activity_7_W_Completeren aanvraag-COMPLETE,Activity_7_W_Completeren aanvraag-SCHEDULE,Activity_7_W_Completeren aanvraag-START,Activity_8_A_ACCEPTED-COMPLETE,Activity_8_A_CANCELLED-COMPLETE,Activity_8_A_DECLINED-COMPLETE,Activity_8_A_FINALIZED-COMPLETE,Activity_8_A_PREACCEPTED-COMPLETE,Activity_8_O_CREATED-COMPLETE,Activity_8_O_SELECTED-COMPLETE,Activity_8_W_Afhandelen leads-COMPLETE,Activity_8_W_Afhandelen leads-SCHEDULE,Activity_8_W_Afhandelen leads-START,Activity_8_W_Beoordelen fraude-COMPLETE,Activity_8_W_Beoordelen fraude-START,Activity_8_W_Completeren aanvraag-COMPLETE,Activity_8_W_Completeren aanvraag-START,Activity_9_A_ACCEPTED-COMPLETE,Activity_9_A_CANCELLED-COMPLETE,Activity_9_A_DECLINED-COMPLETE,Activity_9_A_FINALIZED-COMPLETE,Activity_9_A_PREACCEPTED-COMPLETE,Activity_9_O_SELECTED-COMPLETE,Activity_9_O_SENT-COMPLETE,Activity_9_W_Afhandelen leads-COMPLETE,Activity_9_W_Afhandelen leads-START,Activity_9_W_Beoordelen fraude-COMPLETE,Activity_9_W_Beoordelen fraude-SCHEDULE,Activity_9_W_Beoordelen fraude-START,Activity_9_W_Completeren aanvraag-COMPLETE,Activity_9_W_Completeren aanvraag-SCHEDULE,Activity_9_W_Completeren aanvraag-START,Complete Timestamp,label
0,Variant 613,20000,A_SUBMITTED,COMPLETE,Variant 613,20000,A_PARTLYSUBMITTED,COMPLETE,Variant 613,20000,A_PREACCEPTED,COMPLETE,Variant 613,20000.0,W_Completeren aanvraag,SCHEDULE,Variant 613,20000.0,W_Completeren aanvraag,START,Variant 613,20000.0,A_ACCEPTED,COMPLETE,Variant 613,20000.0,O_SELECTED,COMPLETE

In [3]:
class AccuracyScore:
    def preprocessing(self):
        self['Complete Timestamp'] = pd.to_datetime(self['Complete Timestamp'], format='%Y/%m/%d %H:%M:%S.%f')
        self['Complete Timestamp'] = (self['Complete Timestamp'] - self['Complete Timestamp'].min()) / np.timedelta64(1,'M')
        
        for i in range(0, 10):
            self['Variant_' + str(i)] = self['Variant_' + str(i)].str.replace('Variant', '')
            self['Variant_' + str(i)] = self['Variant_' + str(i)].astype(int)

        ohe = OneHotEncoder(sparse=False)
        for i in range(0, 10):
            self['concept:name_'+str(i)] = ohe.fit_transform(self[['concept:name_'+str(i)]])

        for i in range(0, 10):
            self['lifecycle:transition_'+str(i)] = ohe.fit_transform(self[['lifecycle:transition_'+str(i)]])

        return df
    
    def data_split(self):
        X = self.drop('label', axis=1)
        y = self['label']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023)
        
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        return X_train, y_train, X_test, y_test   

    def imbalance(self, X_train, y_train):
        sm = SMOTE(random_state=1)
        X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
        return X_train_res, y_train_res

    def DT(self, X_train, y_train, X_test, y_test):
        clf_dt = tree.DecisionTreeClassifier(max_depth=2, random_state=2023)
        clf_dt = clf_dt.fit(X_train, y_train)
        predictions = clf_dt.predict(X_test)
        accuracy = clf_dt.score(X_test, y_test)
        print('DT Accuracy:', accuracy)
        print('DT AUC Score {}'.format(roc_auc_score(y_test, predictions)))

    def RF(self, X_train, y_train, X_test, y_test):
        clf_rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=2023)
        clf_rf = clf_rf.fit(X_train, y_train)
        predictions = clf_rf.predict(X_test)
        accuracy = clf_rf.score(X_test, y_test)
        print('RF Accuracy:', accuracy)
        print('RF AUC Score {}'.format(roc_auc_score(y_test, predictions)))
    
    def XGB(self, X_train, y_train, X_test, y_test):
        xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=2023)
        xgb.fit(X_train, y_train)
        xgb_pred = xgb.predict(X_test)

        accuracy = accuracy_score(y_test, xgb_pred)
        precision = precision_score(y_test, xgb_pred)
        recall = recall_score(y_test, xgb_pred)
        f1 = f1_score(y_test, xgb_pred)
        
        print('XGB Accuracy:', accuracy)
        print('XGB AUC Score {}'.format(roc_auc_score(y_test, xgb_pred)))

    def LGBM(self, X_train, y_train, X_test, y_test):
        clf_lgbm = LGBMClassifier(random_state=2023)
        clf_lgbm = clf_lgbm.fit(X_train, y_train)
        predictions = clf_lgbm.predict(X_test)
        accuracy = clf_lgbm.score(X_test, y_test)
        print('LGBM Accuracy:', accuracy)
        print('LGBM AUC Score {}'.format(roc_auc_score(y_test, predictions)))

In [4]:
df = AccuracyScore.preprocessing(df)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_train, y_train, X_test, y_test = AccuracyScore.data_split(df)
X_train, y_train = AccuracyScore.imbalance(df, X_train, y_train)

C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be r

In [5]:
AccuracyScore.DT(df, X_train, y_train, X_test, y_test)
AccuracyScore.RF(df, X_train, y_train, X_test, y_test)
AccuracyScore.XGB(df, X_train, y_train, X_test, y_test)
AccuracyScore.LGBM(df, X_train, y_train, X_test, y_test)

DT Accuracy: 0.8097784568372803
DT AUC Score 0.40499235863474276
RF Accuracy: 0.8329513623631271
RF AUC Score 0.9164544065206317
XGB Accuracy: 0.995161701044054
XGB AUC Score 0.4977075904228222
LGBM Accuracy: 0.9987267634326458
LGBM AUC Score 0.49949057564951604


In [6]:
import numpy as np

unique, counts = np.unique(y_train, return_counts=True)

print(np.asarray((unique, counts)).T)

[[   0 9154]
 [   1 9154]]


In [7]:
y_test.value_counts()

1    3926
0       1
Name: label, dtype: int64